# Đồ án môn học Lập trình song song ứng dụng

# Phát hiện khuôn mặt bằng Cascaded-Classifiers

## Giáo viên: Trần Trung Kiên

## Sinh viên:
- 1712628 - Huỳnh Hiếu Nhân
- 1712657 - Chung Kiệt Phong
- 1712869 - Đào Duy Tuấn

---

# 1. Mô tả ứng dụng

- Nhóm em chọn ứng dụng Phát hiện khuôn mặt bằng Cascaded-Classifiers
	- Input: một tấm ảnh RGB
	- Output: tấm ảnh đầu vào có bounding box ở các khuôn mặt trong ảnh (nếu có)
	- Ý nghĩa của ứng dụng trong thực tế:
		- Hỗ trợ tính năng autofocus trên máy ảnh
		- Là bước quan trọng trong các hệ thống sử dụng nhận diện khuôn mặt: giám sát, an ninh, cơ sở dữ liệu, chăm sóc khách hàng ...
		- Ứng dụng để tự động đề xuất gắn thẻ cho ảnh trên mạng xã hội
- Lý do cần tăng tốc ứng dụng:
	- Có nhiều ý nghĩa thực tế
	- Ứng dụng sẽ chạy chậm nếu cài đặt tuần tự
	- Nhu cầu xử lý dữ liệu lớn, ảnh có độ phân giải cao, video, streaming video trong thời gian thực
- Tiềm năng song song hóa của ứng dụng: 
  - Các khung cửa sổ để phát hiện khuôn mặt ở mọi vị trí trong ảnh và các kích thước khác nhau của khung cửa số xử lý độc lập với nhau
  - Tính tuần tự của các giai đoạn kiểm tra có thách thức lớn nhưng tốc độ xử lý được gia tăng đáng kể nếu có thể song song hóa phần này

---

# 2. Cài đặt tuần tự

## 2.1. Thiết kế

###2.1.1. Khái quát về Haar Cascaded-Classifiers
- Các feature được sử dụng.

- Summed Area Table.

- Thế nào là Cascaded-classifiers ?


####- Haar-like feature



![haar-like-feature.png](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/haar-like-feature.png)
    


> Một số Feature cơ bản:



![haar-features.png](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/haar-features.png)

####- Summed Area Table: dùng để tính nhanh tổng độ xám tại một vị trí (x,y) bất kì





####- Cascaded-classifiers:

![cascaded-classifiers.png](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/cascaded-classifiers.png)



* Trong một bức ảnh, hầu hết là các vùng không phải gương mặt. Vì vậy, ứng với mỗi classifier, ta chỉ cần kiểm tra xem cửa sổ đang xét có phải khuôn mặt hay không. Nếu không phải, loại bỏ cửa sổ ngay mà không cần đến các classifier phía sau. Cửa sổ mà ta đang xét được xác định là khuôn mặt khi nó vượt qua tất cả các classifiers.
* Các features được nhóm thành từng stage khác nhau thay vì dùng tất cả các features để kiểm tra một cửa sổ. Mỗi stage có thể có nhiều hoặc ít features hơn những stages khác. 
* Khi xác định cửa sổ không chứa khuôn mặt, loại bỏ nó ngay và không cần dùng số lượng features còn lại. Điều này giúp giảm thời gian kiểm tra mỗi cửa sổ đi đáng kể. 


###2.1.2. Các bước triển khai
- Input.

- Chuyển ảnh đầu vào sang ảnh xám.

- Đọc mô hình Haar Cascade.

- Tính Summed Area Table (SAT).

- Trượt khung cửa sổ classifier.

- Gom nhóm các khung cửa sổ.

- Output.

####- Input: 
Một tấm ảnh RGB, mô hình Haar Cascade đã train sẵn

#### - Đọc mô hình Haar Cascade: 
Dữ liệu dùng để train: 5000 mẫu positive và 3000 mẫu negative

Gồm nhiều Classifier, mỗi Classifier có một threshold và nhiều Tree, mỗi Tree gồm các node: node trong chứa Feature và threshold; node lá chứa giá trị trả về. Mỗi Feature được định nghĩa bởi hai hoặc ba Rectangle.

Cấu trúc một Classifier:

<img src=https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/classifier.jpg alt="classifier.jpg" width="700"/>


####- Tính Summed Area Table (SAT):
Là một ma trận có kích thước như ảnh đầu vào. Giá trị tại vị trí $(x, y)$ của bảng SAT được tính bằng tổng độ xám của các pixel trong hình chữ nhật có tọa độ 2 góc đối là $(0, 0)$ và $(x, y)$.


> Công thức SAT(x,y) = I(x,y) + SAT(x-1,y) + SAT(x,y-1) - SAT(x-1,y-1)



Sử dụng để tính nhanh Feature:
> SUM(sub-retangle) = SAT(bottom right) - SAT(top right) - SAT(bottom left) + SAT(top left) 

Ảnh minh hoạ:
![summed-area-table.png](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/summed-area-table.png)

####- Trượt khung cửa sổ:
Có kích thước ban đầu là 24x24 trên ảnh xám. Vùng ảnh trong khung cửa sổ được đưa vào Cascaded-Classifiers. Các Tree trong một Classifier sẽ tính Feature mà nó giữ để quyết định giá trị trả về. Classifier lấy tổng giá trị của các Tree, so sánh với threshold để quyết định đẩy vùng ảnh đang xét sang Classifier sau hay loại bỏ. Ảnh cứ thế được lan truyền qua các Classifier cho tới khi bị loại bỏ hoặc ra khỏi Classifier cuối cùng thì vùng ảnh trong khung cửa sổ được ghi nhận lại. Lặp lại quá trình trên với khung cửa sổ có kích thước lớn hơn cho tới tới khi kích thước khung cửa sổ vượt quá kích thước ảnh.


> Minh hoạ:


    
![sliding-dectection-window.gif](https://raw.githubusercontent.com/ddtuan99/parallel-face-detection/main/assets/sliding-dectection-window.gif)

####- Gom nhóm các khung cửa sổ:
Phát hiện cùng một khuôn mặt: hai khung cửa sổ có khoảng cách giữa các cạnh tương ứng đều nhỏ hơn một hằng số $\epsilon$ chỉ định trước thì thuộc cùng một nhóm và ngược lại. Tiếp theo:
	- Loại bỏ các nhóm có số khung cửa sổ thấp hơn ngưỡng chỉ định trước (giảm false positive).
	- Ở mỗi nhóm được giữ lại, tính trung bình tọa độ các khung cửa sổ trong nhóm để gộp các khung thành một bounding box duy nhất và vẽ lên ảnh kết quả.

####- Output: 
Tấm ảnh đầu vào có bounding box ở các khuôn mặt trong ảnh (nếu có)

## 2.2. Đánh giá

- #### Kịch bản thí nghiệm : 
    - Thí nghiệm trên ảnh 501 x 350 có 1 mặt người.

    ![picture](https://drive.google.com/uc?export=view&id=1PyMd0pD-uG4uIJlZ6Szds_xqHrwbyGdg)

    - Thí nghiệm trên ảnh 500 x 768 có 2 mặt người.

    ![picture](https://drive.google.com/uc?export=view&id=1zqxGM0fQtrdzw_djIIOFB-Bp7_fGeO4l)

    - Thí nghiệm trên ảnh 800 x 533 có 6 mặt người.

    ![picture](https://drive.google.com/uc?export=view&id=1BULGaUzD4SeGMWj4sYTLzejv51qBL5Iw)

    - Thí nghiệm trên ảnh 1000 x 666 có 10 mặt người.

    ![picture](https://drive.google.com/uc?export=view&id=1yR2Rik1caiR3o1WuYUYVp-RbfECHoKPi)
  
    - Thí nghiệm trên ảnh 660 x 478 có 29 mặt người.
    ![picture](https://drive.google.com/uc?export=view&id=1f2nUqiQ_gb4imtwnz6jLnF1Iu2ySI_Hg)

- #### Kết quả thí nghiệm: 
    - Kết quả chạy ảnh có 1 mặt người đúng.

    ![picture](https://drive.google.com/uc?export=view&id=1UfrFaAEJGagkZ75SXFSy3JPNYQgCJ-W9)

    - Kết quả chạy ảnh 2 mặt người đúng.

    ![picture](https://drive.google.com/uc?export=view&id=1QEUzwkm_aUmQvCvopZwV2BVCIUyqHy16)

    - Kết quả chạy ảnh 6 mặt người đúng.

    ![picture](https://drive.google.com/uc?export=view&id=1DU1UYGRoXa4SxWGquHmUlSAcCh4DB5ZG)

    - Kết quả chạy ảnh 10 mặt người có một phần đúng và một phần sai.

    ![picture](https://drive.google.com/uc?export=view&id=16r9xUJPw-n-5Zp5neBkNSDc5uH0kAOrn)

    - Kết quả chạy ảnh 29 mặt người đúng.

    ![picture](https://drive.google.com/uc?export=view&id=1J1UADiEyGRON1wLcuVV4bv2m-ZCqKYpn)

 - #### Chạy code :

In [ ]:
!python sequential.py haarcascade_frontalface_default.xml test.jpg result_test.jpg

Jitted vs Numpy  error: 0.0009409751924721985
Jitted vs Opencv error: 0.43670373538637014
Time: 4.023804664611816


In [ ]:
!python sequential.py haarcascade_frontalface_default.xml test6.jpg result_test6.jpg

Jitted vs Numpy  error: 0.00014114114114114113
Jitted vs Opencv error: 0.5132012012012012
Time: 11.12686538696289


In [9]:
!python sequential.py haarcascade_frontalface_default.xml input2.jpg result_output2.jpg

Jitted vs Numpy  error: 0.001421875
Jitted vs Opencv error: 0.4754010416666667
Time: 8.827262163162231


In [10]:
!python sequential.py haarcascade_frontalface_default.xml input6.jpg result_output6.jpg

Jitted vs Numpy  error: 0.00040337711069418386
Jitted vs Opencv error: 0.5025023452157599
Time: 10.643208026885986


In [12]:
!python sequential.py haarcascade_frontalface_default.xml input29.jpg result_output29.jpg

Jitted vs Numpy  error: 0.08250602256878407
Jitted vs Opencv error: 0.05818118422720933
Time: 7.374395132064819


In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2299.998
BogoMIPS:            4599.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs 

---

In [ ]:
from numba import cuda
cuda.detect()

Found 1 CUDA devices
id 0             b'Tesla T4'                              [SUPPORTED]
                      compute capability: 7.5
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True